# Load and Embed PDF artciles
* [Following](https://www.linkedin.com/pulse/build-lightning-fast-rag-chatbot-powered-groqs-lpu-ollama-multani-ssloc)
* [Dockerized chromadb](https://medium.com/@pierrelouislet/getting-started-with-chroma-db-a-beginners-tutorial-6efa32300902)

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True


In [49]:
from langchain_groq import ChatGroq
# from langchain_community.document_loaders import TextLoader, PyPDFDirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores import Chroma
# from langchain_chroma import Chroma
# from langchain_community.embeddings import OllamaEmbeddings
from langchain_community import embeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# from google.colab import userdata
# import os
import time
import textwrap
import gradio as gr
''' fix the problem with sqllite warning '''
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
# 
__groq_api_key__ = os.environ.get("GROQ_API_KEY")

print("All libs loaded!")

All libs loaded!


In [39]:
import os
import sys

proj_dir = os.path.abspath(os.pardir)
sys.path.insert(1,proj_dir.split('wrangler/')[0])
from rezaware.modules.etl.loader import vectorDB as vec
from wrangler.modules.article.stock import vectorize_pdf as vpdf

''' restart initiate classes '''
if debug:
    import importlib
    vec = importlib.reload(vec)
    vpdf= importlib.reload(vpdf)

__desc__ = "read pdf files in folder and store as vectors"
clsPDF = vpdf.dataWorkLoads(desc=__desc__)
__db_type__ = "chromadb"
__chromadb_dir__ = "/home/nuwan/workspace/penman/wrangler/data/article/stock/"
clsVDB = vec.dataWorkLoads(desc=__desc__, db_type=__db_type__, db_root=__chromadb_dir__)

print("\n%s class initialization and load complete!" % __desc__)

All functional VECTORDB-libraries in LOADER-package of ETL-module imported successfully!
All functional VECTORIZE_PDF-libraries in STOCK-package of ARTICLE-module imported successfully!
__propAttr__ Class initialization complete

read pdf files in folder and store as vectors class initialization and load complete!


## Loading documents

In [9]:
data_dir = "/home/nuwan/workspace/penman/wrangler/data/article/stock/cap"
the_text=clsPDF.load_pdf_files(
    folder_path=data_dir
)
print("Loaded %d text pages" % len(the_text))

Loaded 78 text pages


## Splitting Text into Chunks

In [10]:
chunks = clsPDF.text_to_chunks(
    text = the_text
)
print("Split %d docs into %d chunks" % (len(the_text),len(chunks)))

Split 78 docs into 181 chunks


## Setting up the ChromaDB collection

In [46]:
from langchain_community.embeddings import OllamaEmbeddings

# __db_type__ = "chromadb"
__db_name__ = "cap"
# __chromadb_dir__ = "/home/nuwan/workspace/penman/wrangler/data/article/stock/"
__collection__= "ollama_embeds"
_embedding_fn = OllamaEmbeddings(model='nomic-embed-text')

if __collection__ not in [x.name for x in clsVDB.get_collections(db_name = __db_name__)]:
    print("creating document collection %s" % __collection__.upper())
    print(clsVDB.get_collections(db_name = __db_name__))
    vectorstore=clsVDB.store_vectors(
        documents=chunks,
        db_name = __db_name__,
        collection = __collection__,
        embedding_fn=_embedding_fn,
    )
else:
    print("%s collection exists; reading documents" % __collection__.upper())
    vectorstore=clsVDB.read_vectors(
        db_name = __db_name__,
        collection = __collection__,
        embedding_fn=_embedding_fn,        
    )
print("Created %s collection with %d embeddings" 
      % (vectorstore._collection.name.upper(), vectorestore._collection.count()))

OLLAMA_EMBEDS collection exists; reading documents
Created OLLAMA_EMBEDS collection with 181 embeddings


In [47]:
docs = vectorstore.similarity_search("What is CAP?")
docs

[Document(metadata={'page': 1, 'source': '/home/nuwan/workspace/penman/wrangler/data/article/stock/cap/Waidy_S12-strat-opport-challenge_CAP_v20131129.pdf'}, page_content='□Why do we need CAP\n□What is CAP?\n□How do we use it?\n■CAP-Profile\n■Register of Alerting \nAuthorities\n■Multi agency situational \nawareness\n□Automation\n□Pilots\n□Users\n□Conclusion\n□ResourcesOutline'),
 Document(metadata={'page': 6, 'source': '/home/nuwan/workspace/penman/wrangler/data/article/stock/cap/ITU-D_workshop_CAP_report_waidyanatha.pdf'}, page_content='CAP workshop report Version 1.0\n 4.Objectives of the CAP standard\nCAP was developed by Art Botterell et al11 (2006) with an all-hazards all-media approach. A key benefit of CAP  \nfor sending alert messages is that the sender can activate multiple warning systems with a single input. Using a  \nsingle input reduces the cost and complexity of notifying many warning systems. A single input message also  \nprovides consistency in the information delivere

## Setting Up Groq's LPU for Inference

In [50]:
# from langchain_groq import ChatGroq

llm = ChatGroq(
            groq_api_key=__groq_api_key__,
            model_name='mixtral-8x7b-32768'
    )

## Building the RAG Chain

In [51]:
retriever = vectorstore.as_retriever()

rag_template = """Answer this question using the provided context only.
{question}

Context:
{context}"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

## Testing the RAG Architecture

In [52]:
response = rag_chain.invoke("what is this document about?")
print(textwrap.fill(response, width=80))

This document appears to be about the Common Alerting Protocol (CAP) and its
implementation in XML format. According to the document, CAP is a "simple and
general" format for exchanging public warnings and emergencies between alerting
systems. The document provides a detailed description of the various components
of a CAP message, including the alert, info, resource, and area blocks, as well
as the different elements that can be included in each block. The document also
mentions the use of XML schemas for validating the structure of CAP messages.
The document also appears to discuss the use of the Sahana Tools, specifically
the Community Resilience Mapping Tool (CRMT), Alerting and Messaging Broker
(SAMBRO), and Incident Reporting System (IRS), which are developed on the Sahana
Eden Python and Web2Py platform and share common libraries. There is also a
figure that provides an overview of the Alerting Protocol (EDXL-CAP) voice-text
alerts.


## Launching the Gradio Interface

In [41]:
def process_question(user_question):
    # Processing and response time measurement here
    iface = gr.Interface(
        fn=process_question,
        inputs=gr.Textbox(lines=2,
                          placeholder="Type your question here..."),
        outputs=gr.Textbox(),
        title="GROQ CHAT",
        description="Ask any question about your document, and get an answer along with the response time.")
    iface.launch(share=False)

In [42]:
user_question = "what is cap?"
process_question(user_question)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/gradio/queueing.py", line 622, in process_events
    response = await route_utils.call_process_api(
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/gradio/blocks.py", line 2014, in process_api
    result = await self.call_function(
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1567, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/home/nuwan/.cache/pypoetry/virtualenvs/penman-UgVLv5KY-py3.10/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_asy